In [1]:
import dash
import numpy as np
from PIL import Image
from dash import dcc, html, Input, Output
import dash.dependencies as dd
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "INTERACTIVE DATA VISUALIZATION"

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

pos_common = ['top', 'price', 'week', 'next', 'analyst', 'target', 'like', 'burn', 'buy', 'back', 'ust', 'terra', 'money', 'crypto', 'great', 'would', 'yes', 'good', 'market', 'please', 'think', 'let', 'tokens', 'value', 'make', 'happy', 'get', 'way', 'better', 'people', 'still', 'worth', 'could', 'save', 'much', 'new', 'one', 'thank', 'help', 'know', 'hope', 'well', 'btc', 'lost', 'see', 'may', 'come', 'going', 'ecosystem']

neg_common = ['ust', 'buy', 'people', 'crypto', 'terra', 'price', 'bitcoin', 'burn', 'would', 'still', 'back', 'bought', 'kwon', 'see', 'time', 'coins', 'market', 'btc', 'going', 'go', 'lost', 'goes', 'everyone', 'collapse', 'happen', 'crash', 'get', 'think', 'us', 'moon', 'next', 'never', 'make', 'let', 'right', 'got', 'first', 'holders', 'supply', 'new', 'buying', 'way', 'money', 'said', 'one', 'current', 'take', 'token', 'na']

pos = [('top', 31),
 ('price', 30),
 ('week', 30),
 ('next', 27),
 ('analyst', 23),
 ('target', 23),
 ('like', 23),
 ('burn', 20),
 ('buy', 19),
 ('back', 18),
 ('ust', 16),
 ('terra', 12),
 ('money', 11),
 ('crypto', 10),
 ('great', 10),
 ('would', 10),
 ('yes', 9),
 ('good', 9),
 ('market', 9),
 ('think', 8),
 ('let', 8),
 ('tokens', 8),
 ('please', 8),
 ('value', 8),
 ('make', 7),
 ('happy', 7),
 ('get', 7),
 ('way', 7),
 ('better', 7),
 ('people', 7),
 ('still', 7),
 ('worth', 7),
 ('could', 7),
 ('save', 6),
 ('much', 6),
 ('new', 6),
 ('one', 6),
 ('thank', 6),
 ('help', 6),
 ('know', 6),
 ('hope', 6),
 ('well', 6),
 ('btc', 6),
 ('lost', 5),
 ('see', 5),
 ('may', 5),
 ('come', 5),
 ('going', 5),
 ('ecosystem', 5)]

neg = [('ust', 25),
 ('buy', 20),
 ('people', 18),
 ('crypto', 17),
 ('terra', 17),
 ('price', 15),
 ('bitcoin', 15),
 ('burn', 13),
 ('would', 13),
 ('still', 12),
 ('back', 11),
 ('bought', 10),
 ('kwon', 10),
 ('see', 10),
 ('time', 9),
 ('coins', 9),
 ('market', 9),
 ('btc', 9),
 ('go', 9),
 ('lost', 9),
 ('goes', 9),
 ('moon', 8),
 ('everyone', 8),
 ('collapse', 8),
 ('happen', 8),
 ('crash', 8),
 ('going', 8),
 ('get', 8),
 ('think', 8),
 ('us', 8),
 ('next', 7),
 ('buying', 7),
 ('never', 7),
 ('make', 7),
 ('let', 7),
 ('right', 7),
 ('got', 7),
 ('first', 7),
 ('supply', 6),
 ('new', 6),
 ('way', 6),
 ('money', 6),
 ('said', 6),
 ('one', 6),
 ('current', 6),
 ('holders', 6),
 ('take', 6),
 ('token', 5),
 ('na', 5)]

d_pos = dict(pos)
d_neg = dict(neg)

keywords_pos = d_pos.keys()
freq_pos = d_pos.values()

keywords_neg = d_neg.keys()
freq_neg = d_neg.values()

pd_pos = pd.DataFrame()
pd_neg = pd.DataFrame()

pd_pos['keywords'] = keywords_pos
pd_pos['freq'] = freq_pos

pd_neg['keywords'] = keywords_neg
pd_neg['freq'] = freq_neg

In [3]:
# root node
app.layout = html.Div(
    style={
        'height': '98vh',
        'backgroundColor': colors['background']
        },
    children=[
    html.H1(
        children='INTERACTIVE DATA VISUALIZATION WEBAPP',
        style={
            'padding-top': '3vh',
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(
        children='LOGO',
        style={
            'textAlign': 'left',
            'color': colors['text']
        }
    ),

    html.Div(style={
        'margin': 'auto',
        'width': '50%',
        'padding': '10px',
    },
        children=[
            html.P(children=
                   'The price of the LUNA cryptocurrency running on the Terra blockchain has been falling since april. Even more recently, the token has suffered a massive dip resulting in the price falling to aproximately 0.00017 USD per coin. Bellow you can move trough different visualizations related to the LUNA cryptocurrency. The data tell a story:)',
                   style={
                       'color': colors['text']
                       }
                    )
                ]
            ),
        
    # INSTRUCTIONS
    html.Div(style={
                'margin': 'auto',
                'width': '20%',
                'padding': '10px',
                'color': colors['text'],
                'fontSize': '15px',
                'textAlign': 'left',
                'float': 'left',
                'clear': 'left'
            }, 
             children=[
                html.H2('Instructions'),
                html.P('1. Use the slider to change the displayed visualization.'),
                html.P('2. Display the keyword barplot for positive and negative tweets.'),
                html.P('3. Display the keywords as a positive or negative wordcloud.')
            ]
        ),
        
    # INTERACTIVE CONTROLS
    html.Div(children=[
        html.Div(
        dcc.Slider(1, 3, 1,
               value=1,
               id='my-slider'
                ),
        style={
            'margin': 'auto',
            'width': '22%',
            'padding': '10px',
            'float': 'right',
            'clear': 'right'
            }
        ),
        
        html.Div(style={
            'margin': 'auto',
            'width': '22%',
            'padding': '10px',
            'float': 'right',
            'clear': 'right'
                },
            children=[
                    dcc.RadioItems(
                    id='vis_input',
                    options=[
                        {'label': 'Positive tweets', 'value': 'Positive'},
                        {'label': 'Negative tweets', 'value': 'Negative'},
                    ],
                    value='None',
                    labelStyle={
                        'display': 'inline-block',
                        'width': '11em',
                        'line-height': '0.5em'
                        },
                    style={
                        'color': colors['text'],
                        'padding': '10px',
                        'fontSize': '17px'
                            }
                        )
                    ]
                ),  
            ]    
        ),
        
    # THE DATA VISUALIZATION
    html.Div(style={
        'margin': 'auto',
        'width': '50%',
        'height': '50vh',
        'padding': '10px',
        'float': 'center'
            },
        children=html.Div(id='visualization')
        ),
    ]
)


@app.callback(Output("visualization", "children"),
                Input("vis_input", "value"),
                Input("my-slider", "value"))
def update_wordcloud(vis_input, slider_input):
    if slider_input == 1:
        df = pd.read_csv('C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/data/LUNA1-USD.csv')

        fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
        
    elif slider_input == 2 and 'Positive' in vis_input:
        fig = px.bar(pd_pos, x='keywords', y='freq', title="Keywords from positive tweets")
        fig.update_layout(xaxis_tickangle=-45)
    
    elif slider_input == 2 and 'Negative' in vis_input:
        fig = px.bar(pd_neg, x='keywords', y='freq', title="Keywords from negative tweets")
        fig.update_layout(xaxis_tickangle=-45)
        
    elif slider_input == 3 and 'Positive' in vis_input:
        img = np.array(Image.open("C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/plots/pos.png"))
        fig = px.imshow(img)
        fig.update_layout(coloraxis_showscale=False)
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=False)
        
    elif slider_input == 3 and 'Negative' in vis_input:
        img = np.array(Image.open("C:/Users/Rnowa/Documents/interactive-data-visualization-webapp/plots/neg.png"))
        fig = px.imshow(img)
        fig.update_layout(coloraxis_showscale=False)
        fig.update_xaxes(showticklabels=False)
        fig.update_yaxes(showticklabels=False)
    
    return dcc.Graph(figure=fig)
        

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [26/May/2022 15:09:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:04] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:04] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:04] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:04] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2022 15:09:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2022 15:09:04] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "

127.0.0.1 - - [26/May/2022 15:09:07] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/May/2022 15:09:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:09:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:10:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:10:00] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:10:00] "GET /_favicon.ico?v=2.4.1 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:10:00] "GET /_da

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Rnowa\anaconda3\envs\UE_UX_DASH_WEBAPP_WEBSCRAPING\lib\site-packages\dash\dash.py", line 1372, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "

127.0.0.1 - - [26/May/2022 15:10:02] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [26/May/2022 15:10:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:10:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2022 15:10:11] "POST /_dash-update-component HTTP/1.1" 200 -
